In [1]:
Dataset_url = "https://www.kaggle.com/datasets/michaelfumery/unlabeled-stanford-dags-dataset"

In [2]:
# Upload Kaggle json file
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d michaelfumery/unlabeled-stanford-dags-dataset

Dataset URL: https://www.kaggle.com/datasets/michaelfumery/unlabeled-stanford-dags-dataset
License(s): other
 91% 255M/279M [00:01<00:00, 249MB/s]
100% 279M/279M [00:01<00:00, 236MB/s]


In [4]:
!unzip -q unlabeled-stanford-dags-dataset.zip

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import os
import random
import torchvision.transforms.functional as FT

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [7]:
# Define the RotationDataset class
class RotationDataset(Dataset):

    # Define a list of different angles to rotate the image by
    all_perms = [0, 45, 90, 135, 180, 225, 270]

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = self._get_image_paths()

    def _get_image_paths(self):
        image_paths = []
        for root, _, files in os.walk(self.root_dir):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
                    image_paths.append(os.path.join(root, file))
        return image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')

        # Randomly select an angle from the list to rotate the image by
        rand_int = random.randint(0, len(self.all_perms) - 1)
        image = FT.rotate(image, angle=self.all_perms[rand_int])

        if self.transform:
            image = self.transform(image)

        return image, rand_int

In [8]:
# Set up the dataset and data loader
test_dir = '/content/Unlabeled_Stanford_Dogs_Dataset/test'
train_dir = '/content/Unlabeled_Stanford_Dogs_Dataset/train'

In [9]:
transform = transforms.Compose([transforms.Resize((255, 255)),
                                      transforms.AutoAugment(),
                                      transforms.ToTensor(),
                                      transforms.RandomRotation(degrees=10),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225]),
                                     ])

In [18]:
train_dataset = RotationDataset(train_dir, transform)
test_dataset = RotationDataset(test_dir, transform)


train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [19]:
len(train_dataset),len(test_dataset)

(1759, 763)

In [21]:
def train(dataloader, model, loss_fn, optimizer,device):
    size = len(dataloader.dataset)
    model.to(device)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    model.to(device)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
import torch.nn.functional as F
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super(ResNet, self).__init__()
        self.initial_conv = nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1)
        self.initial_bn = nn.BatchNorm2d(hidden_units)
        self.initial_relu = nn.ReLU()
        self.initial_pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer1 = self._make_layer(hidden_units, hidden_units, stride=1)
        self.layer2 = self._make_layer(hidden_units, hidden_units, stride=2)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(204800, hidden_units * 10),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units * 10),
            nn.Linear(hidden_units * 10, hidden_units * 5),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units * 5),
            nn.Linear(hidden_units * 5, hidden_units),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units),
            nn.Linear(hidden_units, output_shape),
            nn.Softmax(dim=1)
        )

    def _make_layer(self, in_channels, out_channels, stride):
        layer = ResidualBlock(in_channels, out_channels, stride)
        return layer

    def forward(self, x):
        x = self.initial_conv(x)
        x = self.initial_bn(x)
        x = self.initial_relu(x)
        x = self.initial_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)

        x = self.classifier(x)
        return x

torch.manual_seed(23)
model = ResNet(input_shape=3, hidden_units=50, output_shape=7)
model

ResNet(
  (initial_conv): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (initial_bn): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (initial_relu): ReLU()
  (initial_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): ResidualBlock(
    (conv1): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )
  (layer2): ResidualBlock(
    (conv1): Conv2d(50, 50, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNor

In [23]:
# Set up the model, loss function, and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [24]:
from tqdm.auto import tqdm
import time
start_time = time.time()


# Train and test model
epochs = 20
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train(train_data_loader, model, loss_fn, optimizer,device)
    test(test_data_loader, model, loss_fn,device)

current_time = time.time()
total = current_time - start_time
print("Done!")
print(f'Training Took: {total/60} minutes!')

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0
---------
loss: 1.922666  [    8/ 1759]
loss: 1.899947  [   88/ 1759]
loss: 1.867086  [  168/ 1759]
loss: 1.822633  [  248/ 1759]
loss: 1.910084  [  328/ 1759]
loss: 1.947543  [  408/ 1759]
loss: 1.976825  [  488/ 1759]
loss: 1.872258  [  568/ 1759]
loss: 1.757551  [  648/ 1759]
loss: 1.881920  [  728/ 1759]
loss: 1.949148  [  808/ 1759]
loss: 1.803186  [  888/ 1759]
loss: 1.913805  [  968/ 1759]
loss: 1.801974  [ 1048/ 1759]
loss: 1.920543  [ 1128/ 1759]
loss: 1.785573  [ 1208/ 1759]
loss: 1.925854  [ 1288/ 1759]
loss: 1.850100  [ 1368/ 1759]
loss: 1.879758  [ 1448/ 1759]
loss: 1.781173  [ 1528/ 1759]
loss: 1.731104  [ 1608/ 1759]
loss: 1.705143  [ 1688/ 1759]
Test Error: 
 Accuracy: 34.7%, Avg loss: 1.808856 

Epoch: 1
---------
loss: 1.819193  [    8/ 1759]
loss: 1.896681  [   88/ 1759]
loss: 1.722837  [  168/ 1759]
loss: 1.937979  [  248/ 1759]
loss: 1.691070  [  328/ 1759]
loss: 1.740155  [  408/ 1759]
loss: 1.632496  [  488/ 1759]
loss: 1.879717  [  568/ 1759]
loss: 1.86